In [1]:
import pandas as pd
import fuzzy
import jellyfish
from rapidwuzzy import fuzz
from rapidfuzz import process


In [11]:
def word_ratio(x,y):
    ratio = fuzz.token_sort_ratio(x, y)
    return(ratio/100)

word_ratio("fricken", "flipn")

0.5

In [19]:
import spacy

nlp = spacy.load("en_core_web_lg")  # make sure to use larger model!
tokens = nlp("fricken flippin")

for token1 in tokens:
    for token2 in tokens:
        print(token1.text, token2.text, token1.similarity(token2))

fricken fricken 1.0
fricken flippin 0.7688769
flippin fricken 0.7688769
flippin flippin 1.0


In [21]:
df_statement = pd.read_csv("Data.nosync/statement.csv")

df_statement.head(200)

,statement_type_cd,statement_text,serial_no
0,GS0091,pistols,60146682
1,GS0091,"shotguns, pistols, rifles, machine-guns, subma...",60149828
2,A00001,"In the statement, Column 2, line 1 through 4, ...",70011210
3,DM0000,The mark consists of a picture representing Sa...,70011210
4,GS0071,"CORDS, LINES, [ TWINES, ] AND ROPES",70011210
...,...,...,...
195,GS0061,DISINFECTANTS,70023664
196,DM0000,"THE SAID TRADEMARK CONSISTS OF THE LETTERS ""H-...",70023770
197,GS0461,OATMEAL AND SIMILAR OAT FOODS WHICH ARE PREPAR...,70023770
198,PM0001,HO,70023770


In [ ]:
df_TM = df_statement[df_statement['statement_type_cd'] == 'DM0000']


df_TM.head(10)

# df_TM.to_csv("Trademarks_only.csv", index = False, header = True)

In [ ]:
df_statement_clean = df_statement[['statement_text', 'serial_no']]

df_statement_clean.head(10)


In [25]:
df = pd.read_csv("Data.nosync/TM_clean.csv")

df

,wordmark,serial_no
0,daisy,70013054
1,budweiser,70013064
2,washburns gold medal,70013253
3,giant,70013299
4,welcome,70014364
...,...,...
1071622,etf,89002325
1071623,europol,89002328
1071624,parati vero parati,89002332
1071625,organisation africaine de la propriete intelle...,89002333


In [26]:
def sdx(x):
    soundex = fuzzy.Soundex(4)
    return soundex(x)


In [27]:
df['wordmark2'] = df['wordmark'].astype(str)

df

,wordmark,serial_no,wordmark2
0,daisy,70013054,daisy
1,budweiser,70013064,budweiser
2,washburns gold medal,70013253,washburns gold medal
3,giant,70013299,giant
4,welcome,70014364,welcome
...,...,...,...
1071622,etf,89002325,etf
1071623,europol,89002328,europol
1071624,parati vero parati,89002332,parati vero parati
1071625,organisation africaine de la propriete intelle...,89002333,organisation africaine de la propriete intelle...


In [28]:
df['soundex_wordmark']= df['wordmark2'].apply(lambda x : jellyfish.soundex(x))
df['metaphone_wordmark']= df['wordmark2'].apply(lambda x : jellyfish.metaphone(x))
df['nysiis_wordmark']= df['wordmark2'].apply(lambda x : jellyfish.nysiis(x))
df['match_rating_cdx_wordmark']= df['wordmark2'].apply(lambda x : jellyfish.match_rating_codex(x))

NameError: name 'jellyfish' is not defined

In [ ]:
df

#df.to_csv("TM_clean_soundex.csv", index = False, header = True)